# g1-4_v034

In [1]:
# import library
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib_venn import venn2, venn2_circles
import seaborn as sns
from tqdm.notebook import tqdm
import pathlib
import plotly
import plotly.express as px
import itertools
import lightgbm as lgb
from optuna.integration import lightgbm as optuna_lgb
import simdkalman
import optuna
import pyproj
from pyproj import Proj, transform
from sklearn import metrics
from sklearn.metrics import roc_curve, precision_recall_curve, confusion_matrix, accuracy_score
pd.set_option('display.max_rows', 100)
import scipy.interpolate
import scipy.sparse

In [2]:
area_labeling = pd.read_csv('../output/prep/area_labeling/result.csv')

g1 = list(area_labeling[area_labeling['g']==1]['collectionName'])
g2 = list(area_labeling[area_labeling['g']==2]['collectionName'])
g3 = list(area_labeling[area_labeling['g']==3]['collectionName'])
g4 = list(area_labeling[area_labeling['g']==4]['collectionName'])
g5 = list(area_labeling[area_labeling['g']==5]['collectionName'])

In [3]:
target = g1 + g2 + g3 + g4

In [4]:
ro_th = 50 # 相対移動距離をもとにした異常値除去の閾値
rog_th = 10 # ground_truthをもとにした異常値除去の閾値

# ground_truthをもとにした異常値除去を行うcollection
rog_target = g3 + g4 + g5

In [5]:
import ipynb_path

def get_nb_name():
    nb_path = ipynb_path.get()
    nb_name = nb_path.rsplit('/',1)[1].replace('.ipynb','')
    return nb_name

In [6]:
# directory setting
nb_name = get_nb_name()
INPUT = '../input/google-smartphone-decimeter-challenge'
OUTPUT = '../output/' + nb_name
os.makedirs(OUTPUT, exist_ok=True)

# utils

In [7]:
def get_train_score(df, gt):
    gt = gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
    df = df.merge(gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
    # calc_distance_error
    df['err'] = calc_haversine(df['latDeg_gt'], df['lngDeg_gt'], df['latDeg'], df['lngDeg'])
    # calc_evaluate_score
    df['phone'] = df['collectionName'] + '_' + df['phoneName']
    res = df.groupby('phone')['err'].agg([percentile50, percentile95])
    res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2 
    score = res['p50_p90_mean'].mean()
    return score

In [8]:
def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

In [9]:
def visualize_trafic(df, center, zoom=9):
    fig = px.scatter_mapbox(df,
                            
                            # Here, plotly gets, (x,y) coordinates
                            lat="latDeg",
                            lon="lngDeg",
                            
                            #Here, plotly detects color of series
                            color="phoneName",
                            labels="phoneName",
                            
                            zoom=zoom,
                            center=center,
                            height=600,
                            width=800)
    fig.update_layout(mapbox_style='stamen-terrain')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(title_text="GPS trafic")
    fig.show()
    
def visualize_collection(df, collection):
    target_df = df[df['collectionName']==collection].copy()
    lat_center = target_df['latDeg'].mean()
    lng_center = target_df['lngDeg'].mean()
    center = {"lat":lat_center, "lon":lng_center}
    
    visualize_trafic(target_df, center)

In [10]:
# ground_truth
def get_ground_truth():
    p = pathlib.Path(INPUT)
    gt_files = list(p.glob('train/*/*/ground_truth.csv'))

    gts = []
    for gt_file in gt_files:
        gts.append(pd.read_csv(gt_file))
    ground_truth = pd.concat(gts)

    return ground_truth

In [11]:
def percentile50(x):
    return np.percentile(x, 50)
def percentile95(x):
    return np.percentile(x, 95)

In [12]:
class train_result:
    def __init__(self, df):
        self.df = df
        self.gt = get_ground_truth()
        self.bl = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
        
        self.gt = self.gt.rename(columns={'latDeg':'latDeg_gt', 'lngDeg':'lngDeg_gt'})
        self.df = self.df.merge(self.gt, on=['collectionName', 'phoneName', 'millisSinceGpsEpoch'], how='inner')
        self.df['phone'] = self.df['collectionName'] + '_' + self.df['phoneName']
        self.df['err'] =  calc_haversine(self.df['latDeg_gt'], self.df['lngDeg_gt'], self.df['latDeg'], self.df['lngDeg'])
        
        self.phone_res = self.calc_err('phone')
        self.clc_res = self.calc_err('collectionName')
        self.phonename_res = self.calc_err('phoneName')
        
    def calc_err(self, by):
        res = self.df.groupby(by)['err'].agg([percentile50, percentile95])
        res['p50_p90_mean'] = (res['percentile50'] + res['percentile95']) / 2
        return res
    
    @property
    def score(self):
        return self.phone_res['p50_p90_mean'].mean()
    @property
    def raw_data(self):
        return self.df
    @property
    def err(self):
        return self.phone_res
    @property
    def collection_err(self):
        return self.clc_res
    @property
    def phonename_err(self):
        return self.phonename_res
    
    def viz_map(self, collection, show_gt=True, show_bl=True):
        tmp = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp2 = self.df[self.df['collectionName']==collection][['collectionName', 'phoneName', 'latDeg_gt', 'lngDeg_gt']]
        tmp2 = tmp2.rename(columns={'latDeg_gt':'latDeg', 'lngDeg_gt':'lngDeg'})
        tmp2['phoneName'] = tmp2['phoneName'] + '_GT'
        tmp3 = self.bl[self.bl['collectionName']==collection][['collectionName', 'phoneName', 'latDeg', 'lngDeg']]
        tmp3['phoneName'] = tmp3['phoneName'] + '_BL'
        
        if show_gt:
            tmp = tmp.append(tmp2)
        if show_bl:
            tmp = tmp.append(tmp3)
        visualize_collection(tmp, collection)

In [13]:
def get_data():
    base_train = pd.read_csv(INPUT + '/' + 'baseline_locations_train.csv')
    base_test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')
    sample_sub = pd.read_csv(INPUT + '/' + 'sample_submission.csv')
    ground_truth = get_ground_truth()
    return base_train, base_test, sample_sub, ground_truth

In [14]:
def update_baseline(df, rb):
    rb = rb.rename(columns={'latDeg':'latDeg_rb', 'lngDeg':'lngDeg_rb'})
    df = df.merge(rb[['millisSinceGpsEpoch', 'phone', 'latDeg_rb', 'lngDeg_rb']], on=['millisSinceGpsEpoch', 'phone'], how='left')

    idx = df[~df['latDeg_rb'].isnull()].index
    df.loc[idx, 'latDeg'] = df.loc[idx, 'latDeg_rb']
    df.loc[idx, 'lngDeg'] = df.loc[idx, 'lngDeg_rb']
    
    df.drop(columns=['latDeg_rb', 'lngDeg_rb'], inplace=True)
    
    return df

# 自前再構成baselineに更新

In [15]:
train, test, sub, gt = get_data()

In [16]:
train = update_baseline(train, pd.read_csv('../output/prep/baseline_g1_v001/result.csv'))
train = update_baseline(train, pd.read_csv('../output/prep/baseline_g2_v003/result.csv'))
train = update_baseline(train, pd.read_csv('../output/prep/baseline_g3_v003/result.csv'))
train = update_baseline(train, pd.read_csv('../output/prep/baseline_g4_v002/result.csv'))

# 相対移動距離をもとにした外れ値除去

In [17]:
def add_distance_diff(df):
    df['latDeg_prev'] = df['latDeg'].shift(1)
    df['latDeg_next'] = df['latDeg'].shift(-1)
    df['lngDeg_prev'] = df['lngDeg'].shift(1)
    df['lngDeg_next'] = df['lngDeg'].shift(-1)
    df['phone_prev'] = df['phone'].shift(1)
    df['phone_next'] = df['phone'].shift(-1)
    
    df['dist_prev'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_prev'], df['lngDeg_prev'])
    df['dist_next'] = calc_haversine(df['latDeg'], df['lngDeg'], df['latDeg_next'], df['lngDeg_next'])
    
    df.loc[df['phone']!=df['phone_prev'], ['latDeg_prev', 'lngDeg_prev', 'dist_prev']] = np.nan
    df.loc[df['phone']!=df['phone_next'], ['latDeg_next', 'lngDeg_next', 'dist_next']] = np.nan
    
    return df

In [18]:
# reject outlier
train = add_distance_diff(train)
train.loc[((train['dist_prev'] > ro_th) & (train['dist_next'] > ro_th)), ['latDeg', 'lngDeg']] = np.nan
train.to_csv(OUTPUT + '/train_ro.csv', index=False)

# ground_truthを基準にした外れ値除去

In [19]:
def get_osmnx_data():
    p = pathlib.Path(INPUT)
    files = list(p.glob('prep/osmnx/*.csv'))

    dfs = []
    for file in files:
        dfs.append(pd.read_csv(file))
    osmnx = pd.concat(dfs)

    return osmnx

In [20]:
def remove_based_on_gt(target_df, target_gt, th):
    osmnx_data = get_osmnx_data()
    target_gt = target_gt.append(osmnx_data)
    
    for idx in target_df.index:
        lat = target_df.at[idx, 'latDeg']
        lng = target_df.at[idx, 'lngDeg']
        collection = target_df.at[idx, 'collectionName']
        
        if np.isnan(lat):
            continue
        else:
            target_gt['latDeg_pred'] = lat
            target_gt['lngDeg_pred'] = lng
            target_gt['dist'] = calc_haversine(target_gt['latDeg'], target_gt['lngDeg'], target_gt['latDeg_pred'], target_gt['lngDeg_pred'])
            closest_dist = target_gt['dist'].min()
            if closest_dist > th:
                target_df.at[idx, 'latDeg'] = np.nan
                target_df.at[idx, 'lngDeg'] = np.nan
    return target_df

In [21]:
def interpolate_remove_point(df):
    out_df = pd.DataFrame()
    collections = df['collectionName'].unique()
    
    for collection in collections:
        tmp = df[df['collectionName']==collection].copy()
        tmp = tmp.sort_values('millisSinceGpsEpoch')
        tmp = tmp.reset_index().set_index('millisSinceGpsEpoch')
        tmp[['latDeg', 'lngDeg']] = tmp[['latDeg', 'lngDeg']].interpolate(method='index', limit_area='inside')
        tmp = tmp.sort_values('index')
        tmp = tmp.reset_index().set_index('index')
        out_df = out_df.append(tmp)
    return out_df

In [22]:
def interpolate_by_rel(df):
    df_idx = list(df.index)
    df['lat_interp_f'] = df['latDeg']
    df['lng_interp_f'] = df['lngDeg']
    df['lat_interp_b'] = df['latDeg']
    df['lng_interp_b'] = df['lngDeg']
    
    for idx in df_idx:
        if idx == df_idx[0]:
            continue
        if df.at[idx, 'phone'] != df.at[idx-1, 'phone']:
            continue
        if ~np.isnan(df.at[idx, 'latDeg']):
            continue
            
        lat_prev = df.at[idx-1, 'lat_interp_f']
        lng_prev = df.at[idx-1, 'lng_interp_f']
        lat_diff = df.at[idx-1, 'lat_diff']
        lng_diff = df.at[idx-1, 'lng_diff']
        
        if ~np.isnan(lat_prev) & ~np.isnan(lat_diff):
            df.at[idx, 'lat_interp_f'] = lat_prev + lat_diff
            df.at[idx, 'lng_interp_f'] = lng_prev + lng_diff
    
    df_idx.reverse()
    for idx in df_idx:
        if idx == df_idx[0]:
            continue
        if df.at[idx, 'phone'] != df.at[idx+1, 'phone']:
            continue
        if ~np.isnan(df.at[idx, 'latDeg']):
            continue
            
        lat_prev = df.at[idx+1, 'lat_interp_b']
        lng_prev = df.at[idx+1, 'lng_interp_b']
        lat_diff = df.at[idx+1, 'lat_diff']
        lng_diff = df.at[idx+1, 'lng_diff']
        
        if ~np.isnan(lat_prev) & ~np.isnan(lat_diff):
            df.at[idx, 'lat_interp_b'] = lat_prev - lat_diff
            df.at[idx, 'lng_interp_b'] = lng_prev - lng_diff
            
    df['latDeg'] = (df['lat_interp_f'] + df['lat_interp_b']) / 2
    df['lngDeg'] = (df['lng_interp_f'] + df['lng_interp_b']) / 2
            
    return df

In [23]:
rog_df = train[train['collectionName'].isin(rog_target)].copy()
rog_gt = gt[gt['collectionName'].isin(rog_target)].copy()
rog_df = remove_based_on_gt(rog_df, rog_gt, rog_th)
train.loc[rog_df.index, ['latDeg', 'lngDeg']] = rog_df[['latDeg', 'lngDeg']]

In [24]:
train_sp0_pred = pd.read_csv('../output/prep/speed0_pred_v001/train_sp0_pred.csv')
dp_train = pd.read_csv('../output/prep/rel_pred_v001/train_result.csv')

train = train.merge(train_sp0_pred[['phone', 'millisSinceGpsEpoch', 'isSpeed0']], on=['phone', 'millisSinceGpsEpoch'], how='left')
train = train.merge(dp_train[['millisSinceGpsEpoch', 'phone', 'lat_diff', 'lng_diff']], on=['millisSinceGpsEpoch', 'phone'], how='left')
train.loc[train['isSpeed0']==1, ['lat_diff', 'lng_diff']] = 0

In [25]:
train = interpolate_by_rel(train)
train = interpolate_remove_point(train)

In [26]:
res = train_result(train[train['collectionName'].isin(target)])
print(res.score)
display(res.err)

3.8004216748626285


,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.116276,2.400690,1.758483
2020-05-14-US-MTV-1_Pixel4XLModded,1.691268,3.686179,2.688723
2020-05-14-US-MTV-2_Pixel4,1.523077,2.728798,2.125938
2020-05-14-US-MTV-2_Pixel4XLModded,3.379125,10.002392,6.690758
2020-05-21-US-MTV-1_Pixel4,1.943373,4.794928,3.369151
2020-05-21-US-MTV-2_Pixel4,0.898226,2.804577,1.851401
2020-05-21-US-MTV-2_Pixel4XL,1.085159,3.418578,2.251868
2020-05-29-US-MTV-1_Pixel4,2.485724,4.288529,3.387127
2020-05-29-US-MTV-1_Pixel4XL,2.235506,3.395295,2.815400


# kalmanフィルタ

In [27]:
T = 1.0
state_transition = np.array([[1, 0, T, 0, 0.5 * T ** 2, 0], [0, 1, 0, T, 0, 0.5 * T ** 2], [0, 0, 1, 0, T, 0],
                             [0, 0, 0, 1, 0, T], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1]])
process_noise = np.diag([1e-5, 1e-5, 5e-6, 5e-6, 1e-6, 1e-6]) + np.ones((6, 6)) * 1e-9
observation_model = np.array([[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]])
observation_noise = np.diag([5e-5, 5e-5]) + np.ones((2, 2)) * 1e-9

kf = simdkalman.KalmanFilter(
        state_transition = state_transition,
        process_noise = process_noise,
        observation_model = observation_model,
        observation_noise = observation_noise)

def apply_kf_smoothing(df, kf_=kf):
    unique_paths = df[['collectionName', 'phoneName']].drop_duplicates().to_numpy()
    for collection, phone in unique_paths:
        cond = np.logical_and(df['collectionName'] == collection, df['phoneName'] == phone)
        data = df[cond][['latDeg', 'lngDeg']].to_numpy()
        data = data.reshape(1, len(data), 2)
        smoothed = kf_.smooth(data)
        df.loc[cond, 'latDeg'] = smoothed.states.mean[0, :, 0]
        df.loc[cond, 'lngDeg'] = smoothed.states.mean[0, :, 1]
    return df

In [28]:
train = apply_kf_smoothing(train)

# speed0の処理

In [29]:
def sp0_process(df):
    df['group'] = df.groupby('phone').cumcount()
    df.loc[(df['group']>0)&(df['isSpeed0']==1),'group'] = np.nan
    df['group'] = df['group'].fillna(method='ffill')
    df[['latDeg', 'lngDeg']] = df.groupby(['phone', 'group'])[['latDeg','lngDeg']].transform('mean')
    return df

In [30]:
train = sp0_process(train)

# cost_minimization

In [31]:
def cost_minimization(df, b):
    xy_hat = df[['latDeg', 'lngDeg']].to_numpy()
    delta_xy_hat = df[['lat_diff', 'lng_diff']].fillna(method='ffill')[:-1].to_numpy()

    N = xy_hat.shape[0]
    alpha = 0.01 * np.ones(N)
    beta  = b * np.ones(N-1)

    A = scipy.sparse.spdiags(alpha, [0], N, N)
    B = scipy.sparse.spdiags(beta, [0], N-1, N-1)
    D = scipy.sparse.spdiags(np.stack([-np.ones(N), np.ones(N)]), [0, 1], N-1, N)

    Q = A + (D.T @ B @ D)
    c = (A @ xy_hat) + (D.T @ (B @ delta_xy_hat))
    xy_star = scipy.sparse.linalg.spsolve(Q, c)

    return xy_star

In [32]:
def cost_min_objective(trial):
    cost_min_beta_g1 = trial.suggest_uniform('cost_min_beta_g1', 0.01, 0.5)
    cost_min_beta_g2 = trial.suggest_uniform('cost_min_beta_g2', 0.01, 0.5)
    cost_min_beta_g3 = trial.suggest_uniform('cost_min_beta_g3', 0.01, 0.5)
    cost_min_beta_g4 = trial.suggest_uniform('cost_min_beta_g4', 0.01, 0.5)
    train_ = train.copy()
    
    for g, w in zip([g1,g2,g3,g4],[cost_min_beta_g1, cost_min_beta_g2, cost_min_beta_g3, cost_min_beta_g4]):
        phones = train[(train['collectionName'].isin(g))&(train['phoneName']!='Mi8')]['phone'].unique()
        for phone in phones:
            idx = train[train['phone']==phone].index
            train_.loc[idx, ['latDeg', 'lngDeg']] = cost_minimization(train_.loc[idx], w)
    score = train_result(train_[train_['collectionName'].isin(target)]).score
    return score

In [33]:
study = optuna.create_study()
study.optimize(cost_min_objective, n_trials=100)
print(study.best_params)

cost_min_beta_g1 = study.best_params['cost_min_beta_g1']
cost_min_beta_g2 = study.best_params['cost_min_beta_g2']
cost_min_beta_g3 = study.best_params['cost_min_beta_g3']
cost_min_beta_g4 = study.best_params['cost_min_beta_g4']

[I 2021-08-01 20:06:44,647] A new study created in memory with name: no-name-4788a85e-e959-4694-9761-2a9e10ca2813
[I 2021-08-01 20:06:45,787] Trial 0 finished with value: 2.8839473096103236 and parameters: {'cost_min_beta_g1': 0.2327117088502359, 'cost_min_beta_g2': 0.4104625977714043, 'cost_min_beta_g3': 0.4451115916395785, 'cost_min_beta_g4': 0.40805718126212454}. Best is trial 0 with value: 2.8839473096103236.
[I 2021-08-01 20:06:47,021] Trial 1 finished with value: 2.856089879281912 and parameters: {'cost_min_beta_g1': 0.19808248359929345, 'cost_min_beta_g2': 0.2067932489552257, 'cost_min_beta_g3': 0.25510554859369944, 'cost_min_beta_g4': 0.24178748840086547}. Best is trial 1 with value: 2.856089879281912.
[I 2021-08-01 20:06:48,210] Trial 2 finished with value: 2.835336829858217 and parameters: {'cost_min_beta_g1': 0.07105599940448336, 'cost_min_beta_g2': 0.27513001719967695, 'cost_min_beta_g3': 0.1437129376972357, 'cost_min_beta_g4': 0.390356207682572}. Best is trial 2 with value

{'cost_min_beta_g1': 0.0535372451226281, 'cost_min_beta_g2': 0.08437882564958417, 'cost_min_beta_g3': 0.4978149154635596, 'cost_min_beta_g4': 0.4515632530829281}


In [34]:
for g, w in zip([g1,g2,g3,g4],[cost_min_beta_g1, cost_min_beta_g2, cost_min_beta_g3, cost_min_beta_g4]):
    phones = train[(train['collectionName'].isin(g))&(train['phoneName']!='Mi8')]['phone'].unique()
    for phone in phones:
        idx = train[train['phone']==phone].index
        train.loc[idx, ['latDeg', 'lngDeg']] = cost_minimization(train.loc[idx], w)
print(train_result(train[train['collectionName'].isin(target)]).score)

2.7957393615904618


# phones_weighted_mean

In [35]:
train_ = train[train['collectionName'].isin(target)].copy()

In [36]:
def calc_mean_pred(df, ws):
    '''
    Generate interpolated lat,lng values for different phone times in the same collection.
    '''
    time_list = df[['collectionName', 'millisSinceGpsEpoch']].drop_duplicates()
    phone_list = df[['collectionName', 'phoneName']].drop_duplicates()
    tmp = time_list.merge(phone_list, on='collectionName', how='outer')
    tmp['phone'] = tmp['collectionName'] + '_' + tmp['phoneName']
    tmp = tmp.merge(df, on=['collectionName', 'phoneName', 'phone', 'millisSinceGpsEpoch'], how='left')
    
    lerp_df = pd.DataFrame()
    for phone in tmp['phone'].unique():
        lerp_tmp = tmp[tmp['phone']==phone].copy()
        lerp_tmp = lerp_tmp.sort_values('millisSinceGpsEpoch')
        lerp_tmp = lerp_tmp.set_index('millisSinceGpsEpoch')
        lerp_tmp[['latDeg', 'lngDeg']] = lerp_tmp[['latDeg', 'lngDeg']].interpolate(method='index', limit_area='inside')
        lerp_tmp = lerp_tmp.reset_index()
        lerp_df = lerp_df.append(lerp_tmp)
    
    lerp_df['w'] = lerp_df['phoneName'].replace(ws)
    lerp_df.loc[lerp_df['latDeg'].isnull(), 'w'] = np.nan
    lerp_df['w_sum'] = lerp_df.groupby(['collectionName', 'millisSinceGpsEpoch'])['w'].transform('sum')
    lerp_df['w'] = lerp_df['w'] / lerp_df['w_sum']
    lerp_df['latDeg'] = lerp_df['latDeg'] * lerp_df['w']
    lerp_df['lngDeg'] = lerp_df['lngDeg'] * lerp_df['w']
    mean_pred = lerp_df.groupby(['collectionName', 'millisSinceGpsEpoch'])['latDeg', 'lngDeg'].sum().reset_index()
    
    output_df = df[['collectionName', 'phoneName', 'millisSinceGpsEpoch']].copy()
    output_df = output_df.merge(mean_pred[['collectionName', 'millisSinceGpsEpoch', 'latDeg', 'lngDeg']], on=['collectionName', 'millisSinceGpsEpoch'], how='left')
    return output_df

In [37]:
def phones_mean_objective(trial):
    Pixel4 = trial.suggest_int('Pixel4', 1, 10)
    Pixel4XLModded = trial.suggest_int('Pixel4XLModded', 1, 10)
    Pixel4XL = trial.suggest_int('Pixel4XL', 1, 10)
    Mi8 = trial.suggest_int('Mi8', 1, 10)
    Pixel4Modded = trial.suggest_int('Pixel4Modded', 1, 10)
    Pixel5 = trial.suggest_int('Pixel5', 1, 10)
    SamsungS20Ultra = trial.suggest_int('SamsungS20Ultra', 1, 10)
    
    ws = {'Pixel4':Pixel4, 'Pixel4XLModded':Pixel4XLModded, 'Pixel4XL':Pixel4XL, 'Mi8':Mi8, 'Pixel4Modded':Pixel4Modded,
          'Pixel5':Pixel5, 'SamsungS20Ultra':SamsungS20Ultra}
    
    out = calc_mean_pred(train_, ws)
    score = train_result(out).score
    
    return score

In [38]:
study = optuna.create_study()
study.optimize(phones_mean_objective, n_trials=100)
phones_mean_weights = study.best_params
print(phones_mean_weights)

[I 2021-08-01 20:08:43,925] A new study created in memory with name: no-name-843fe5d7-fdcd-4a53-8504-8f31eb8bfe02
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
[I 2021-08-01 20:08:48,611] Trial 0 finished with value: 2.399397202738301 and parameters: {'Pixel4': 8, 'Pixel4XLModded': 8, 'Pixel4XL': 3, 'Mi8': 10, 'Pixel4Modded': 6, 'Pixel5': 8, 'SamsungS20Ultra': 9}. Best is trial 0 with value: 2.399397202738301.
[I 2021-08-01 20:08:53,193] Trial 1 finished with value: 2.5535158381258087 and parameters: {'Pixel4': 4, 'Pixel4XLModded': 10, 'Pixel4XL': 1, 'Mi8': 8, 'Pixel4Modded': 2, 'Pixel5': 3, 'SamsungS20Ultra': 10}. Best is trial 0 with value: 2.399397202738301.
[I 2021-08-01 20:08:57,817] Trial 2 finished with value: 2.4661013030323113 and parameters: {'Pixel4': 8, 'Pixel4XLModded': 5, 'Pixel4XL': 10, 'Mi8': 3, 'Pixel4Modded': 1, 'Pixel5': 10, 

{'Pixel4': 4, 'Pixel4XLModded': 6, 'Pixel4XL': 2, 'Mi8': 5, 'Pixel4Modded': 10, 'Pixel5': 4, 'SamsungS20Ultra': 1}


In [39]:
train = calc_mean_pred(train, phones_mean_weights)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


# position_shift

In [40]:
train['phone'] = train['collectionName'] + '_' + train['phoneName']

In [41]:
def WGS84_to_ECEF(lat, lon, alt):
    # convert to radians
    rad_lat = lat * (np.pi / 180.0)
    rad_lon = lon * (np.pi / 180.0)
    a    = 6378137.0
    # f is the flattening factor
    finv = 298.257223563
    f = 1 / finv   
    # e is the eccentricity
    e2 = 1 - (1 - f) * (1 - f)    
    # N is the radius of curvature in the prime vertical
    N = a / np.sqrt(1 - e2 * np.sin(rad_lat) * np.sin(rad_lat))
    x = (N + alt) * np.cos(rad_lat) * np.cos(rad_lon)
    y = (N + alt) * np.cos(rad_lat) * np.sin(rad_lon)
    z = (N * (1 - e2) + alt)        * np.sin(rad_lat)
    return x, y, z

transformer = pyproj.Transformer.from_crs(
    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},)

def ECEF_to_WGS84(x,y,z):
    lon, lat, alt = transformer.transform(x,y,z,radians=False)
    return lon, lat, alt
msge = 'millisSinceGpsEpoch'

In [42]:
def position_shift(df,a):

    d = df.copy()
    d['heightAboveWgs84EllipsoidM'] = 63.5
    d['x'], d['y'], d['z'] = zip(*d.apply(lambda x: WGS84_to_ECEF(x.latDeg, x.lngDeg, x.heightAboveWgs84EllipsoidM), axis=1))

    #a = -0.2
    d.sort_values(['phone', msge], inplace=True)
    for fi in ['x','y','z']:
        d[[fi+'p']] = d[fi].shift().where(d['phone'].eq(d['phone'].shift()))
        d[[fi+'diff']] = d[fi]-d[fi+'p']
    #d[['yp']] = d['y'].shift().where(d['phone'].eq(d['phone'].shift()))
    d[['dist']] = np.sqrt(d['xdiff']**2 + d['ydiff']**2+ d['zdiff']**2)
    for fi in ['x','y','z']:
        d[[fi+'new']] = d[fi+'p'] + d[fi+'diff']*(1-a/d['dist'])
    lng, lat, alt = ECEF_to_WGS84(d['xnew'].values,d['ynew'].values,d['znew'].values)
    
    lng[np.isnan(lng)] = d.loc[np.isnan(lng),'lngDeg']
    lat[np.isnan(lat)] = d.loc[np.isnan(lat),'latDeg']
    d['latDeg'] = lat
    d['lngDeg'] = lng
    
    return d 

def objective(trial):
    a = trial.suggest_uniform('a', -1, 1)
    score = get_train_score(position_shift(train, a),gt)
    return score

In [43]:
study = optuna.create_study()
study.optimize(objective, n_trials=30)
opt_a = study.best_params['a']
print(opt_a)

[I 2021-08-01 20:16:26,595] A new study created in memory with name: no-name-fea2d234-32c0-449d-82f1-7853c4fad7b0
[I 2021-08-01 20:16:30,475] Trial 0 finished with value: 3.0106162232984395 and parameters: {'a': 0.84022226730049}. Best is trial 0 with value: 3.0106162232984395.
[I 2021-08-01 20:16:34,360] Trial 1 finished with value: 3.2276588791679504 and parameters: {'a': -0.583534077592049}. Best is trial 0 with value: 3.0106162232984395.
[I 2021-08-01 20:16:38,226] Trial 2 finished with value: 3.0722198356261314 and parameters: {'a': -0.2739728591962798}. Best is trial 0 with value: 3.0106162232984395.
[I 2021-08-01 20:16:41,951] Trial 3 finished with value: 2.989075565362268 and parameters: {'a': 0.736854166947658}. Best is trial 3 with value: 2.989075565362268.
[I 2021-08-01 20:16:45,899] Trial 4 finished with value: 3.078066424768502 and parameters: {'a': -0.28866641294696094}. Best is trial 3 with value: 2.989075565362268.
[I 2021-08-01 20:16:49,651] Trial 5 finished with value

0.4362453399364912


In [44]:
train = position_shift(train, opt_a)
train.to_csv(OUTPUT + '/train_ro_rog_kf_sp0_pm_ps.csv', index=False)

In [45]:
train = train[train['collectionName'].isin(target)].copy()

# trainの結果確認

In [46]:
result = train_result(train)

In [47]:
result.score

2.212895160416191

In [48]:
result.err

,percentile50,percentile95,p50_p90_mean
phone,,,
2020-05-14-US-MTV-1_Pixel4,1.091003,2.046768,1.568885
2020-05-14-US-MTV-1_Pixel4XLModded,0.984478,1.896169,1.440323
2020-05-14-US-MTV-2_Pixel4,1.890439,2.993001,2.441720
2020-05-14-US-MTV-2_Pixel4XLModded,1.971189,3.643106,2.807148
2020-05-21-US-MTV-1_Pixel4,2.024545,3.567803,2.796174
2020-05-21-US-MTV-2_Pixel4,0.635783,2.157634,1.396708
2020-05-21-US-MTV-2_Pixel4XL,0.592976,2.085873,1.339424
2020-05-29-US-MTV-1_Pixel4,1.617745,2.562769,2.090257
2020-05-29-US-MTV-1_Pixel4XL,1.586823,2.561072,2.073947


In [49]:
train.to_csv(OUTPUT + f'/{nb_name}_train.csv', index=False)

# make_sub

In [50]:
test = pd.read_csv(INPUT + '/' + 'baseline_locations_test.csv')

In [51]:
test = update_baseline(test, pd.read_csv('../output/prep/baseline_g1_v001/result_test.csv'))
test = update_baseline(test, pd.read_csv('../output/prep/baseline_g2_v003/result_test.csv'))
test = update_baseline(test, pd.read_csv('../output/prep/baseline_g3_v003/result_test.csv'))
test = update_baseline(test, pd.read_csv('../output/prep/baseline_g4_v002/result_test.csv'))

In [52]:
test = add_distance_diff(test)
test.loc[((test['dist_prev'] > ro_th) & (test['dist_next'] > ro_th)), ['latDeg', 'lngDeg']] = np.nan

In [53]:
rog_df = test[test['collectionName'].isin(rog_target)].copy()
rog_gt = gt[gt['collectionName'].isin(rog_target)].copy()
rog_df = remove_based_on_gt(rog_df, rog_gt, rog_th)
rog_df.to_csv(OUTPUT + '/test_rog.csv', index=False)
test.loc[rog_df.index, ['latDeg', 'lngDeg']] = rog_df[['latDeg', 'lngDeg']]

In [54]:
test = interpolate_by_rel(test)
test = interpolate_remove_point(test)

KeyError: 'lat_diff'

In [ ]:
test_sp0_pred = pd.read_csv('../output/prep/speed0_pred_v001/test_sp0_pred.csv')
dp_test = pd.read_csv('../output/prep/rel_pred_v001/test_result.csv')

test = test.merge(test_sp0_pred[['phone', 'millisSinceGpsEpoch', 'isSpeed0']], on=['phone', 'millisSinceGpsEpoch'], how='left')
test = test.merge(dp_test[['millisSinceGpsEpoch', 'phone', 'lat_diff', 'lng_diff']], on=['millisSinceGpsEpoch', 'phone'], how='left')
test.loc[test['isSpeed0']==1, ['lat_diff', 'lng_diff']] = 0

In [ ]:
test = apply_kf_smoothing(test)

In [ ]:
test_sp0_pred = pd.read_csv('../output/prep/speed0_pred_v001/test_sp0_pred.csv')
test = sp0_process(test, test_sp0_pred)

In [ ]:
dp_test = pd.read_csv('../output/prep/rel_pred_v001/test_result.csv')
test = test.merge(dp_test[['millisSinceGpsEpoch', 'phone', 'lat_diff', 'lng_diff']], on=['millisSinceGpsEpoch', 'phone'], how='left')
test.loc[test['isSpeed0']==1, ['lat_diff', 'lng_diff']] = 0

for g, w in zip([g1,g2,g3,g4],[cost_min_beta_g1, cost_min_beta_g2, cost_min_beta_g3, cost_min_beta_g4]):
    phones = test[(test['collectionName'].isin(g))&(test['phoneName']!='Mi8')]['phone'].unique()
    for phone in phones:
        idx = test[test['phone']==phone].index
        test.loc[idx, ['latDeg', 'lngDeg']] = cost_minimization(test.loc[idx], w)

In [ ]:
test = calc_mean_pred(test, phones_mean_weights)

In [ ]:
test['phone'] = test['collectionName'] + '_' + test['phoneName']

In [ ]:
test = position_shift(test, opt_a)

In [ ]:
test = test[test['collectionName'].isin(target)]

In [ ]:
test.to_csv(OUTPUT + f'/{nb_name}_test.csv', index=False)